In [1]:
using CSV, DataFrames

# Uploading CPS data

In [2]:
cps = DataFrame(CSV.File("c:\\data\\Bounds\\cps.csv")) # <-change this to the right directory

## Cleaning zero wage

cps = cps[cps.wage .> 0, :]

(22715, 3)

In [3]:
first(cps,5)

,age,wage,educ
,Int64,Int64,String
1,22,12000,some college but no degree
2,21,3500,some college but no degree
3,49,30000,some college but no degree
4,31,32000,bachelor's degree
5,42,89630,doctorate degree


In [4]:
# Numerical value for education

#1.Creating a dictionary

educ_key = unique(cps.educ);
educ_val = [13,14,16,21,12,14,14,13,10,12,11,9,8,6,4,0];

educ_dict = Dict()
for (k,v) in zip(educ_key,educ_val)
    push!(educ_dict, k =>v)
end
educ_dict

Dict{Any, Any} with 16 entries:
  "grade 11"                                            => 10
  "some college but no degree"                          => 13
  "associate's degree, academic program"                => 13
  "grade 10"                                            => 9
  "grades 7 or 8"                                       => 8
  "grades 1, 2, 3, or 4"                                => 4
  "associate's degree, occupational/vocational program" => 12
  "high school diploma or equivalent"                   => 21
  "grade 9"                                             => 11
  "none or preschool"                                   => 0
  "doctorate degree"                                    => 16
  "bachelor's degree"                                   => 14
  "master's degree"                                     => 14
  "grades 5 or 6"                                       => 6
  "professional school degree"                          => 14
  "12th grade, no diploma"                 

In [5]:
#2. transforming cps.educ
f(x) = educ_dict[x]
cps.educ_num = f.(cps.educ)

#3. log of wage
cps.log_wage = log.(cps.wage)
first(cps,10)

,age,wage,educ,educ_num,log_wage
,Int64,Int64,String,Int64,Float64
1,22,12000,some college but no degree,13,9.39266
2,21,3500,some college but no degree,13,8.16052
3,49,30000,some college but no degree,13,10.309
4,31,32000,bachelor's degree,14,10.3735
5,42,89630,doctorate degree,16,11.4034
6,35,229339,doctorate degree,16,12.343
7,42,39000,high school diploma or equivalent,21,10.5713
8,48,50000,high school diploma or equivalent,21,10.8198
9,41,37500,some college but no degree,13,10.5321


In [7]:
Nobs, = size(cps)

(22715, 5)